In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import random
import pandas as pd
from torch import nn
from glob import glob
from tqdm.auto import tqdm
from torchaudio import transforms as T
import pytorch_lightning as pl 
from maatool.data.feats_itdataset_v2 import FeatsIterableDatasetV2
from maatool.models.transformer import TransformerWithSinPos
from maatool.models.conformer import ConformerWithSinPos
from copy import deepcopy
torch.cuda.is_available()

In [ ]:
import logging
import logging.config

def configure_logging(log_level):
    handlers =  {
            "maa": {
                "class": "logging.StreamHandler",
                "formatter": "maa_basic",
                "stream": "ext://sys.stdout",
            }
    }
    CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {"maa_basic": {"format": '%(asctime)s %(name)s %(pathname)s:%(lineno)d - %(levelname)s - %(message)s'}},
        "handlers": handlers,
        "loggers": {"maa": {"handlers": handlers.keys(), "level": log_level}},
        "root": {"handlers": handlers.keys(), "level": log_level}
    }
    logging.config.dictConfig(CONFIG)
configure_logging("INFO")

In [ ]:
from collections import defaultdict

In [ ]:
from maatool.lightning.swipe_recognizer import SwipeTransformerRecognizer

In [ ]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)

v5_ckpt = 'exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt'
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(v5_ckpt,
                                                            backbone=model, 
                                                            map_location='cpu')


In [ ]:
val_ds = FeatsIterableDatasetV2([f"ark:data_feats/valid/feats.ark"], 
                             targets_rspecifier='ark:exp/bpe500/valid-text.int', 
                                shuffle=False,
                               bos_id=1, 
                               eos_id=2,
                               batch_first=False)
val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=1, collate_fn=val_ds.collate_pad)


In [ ]:
trainer = pl.Trainer(callbacks=[pl.callbacks.TQDMProgressBar(refresh_rate=100)])

In [ ]:
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)

v12_ckpt = 'exp/models/t_finetune_with_sa/lightning_logs/version_50454224/checkpoints/epoch=2-step=70000.ckpt'
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(v12_ckpt,
                                                            backbone=model, 
                                                            map_location='cpu')

In [ ]:
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
model = ConformerWithSinPos(feats_dim=37, num_tokens=500, num_decoder_layers=8, num_encoder_layers=8)
#v_13_ckpt = 'exp/models/conformer_v1/lightning_logs/version_50454211/checkpoints/epoch=0-step=60000.ckpt'
v_15_ckpt = 'exp/models/conformer_v1/lightning_logs/version_50454211/checkpoints/last.ckpt'
v_16_ckpt = 'exp/models/conformer_v1/lightning_logs/version_50454211/checkpoints/epoch=1-step=80000.ckpt'

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(v_16_ckpt,
                                                            backbone=model, 
                                                            map_location='cpu')

In [ ]:
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
model_v15 = SwipeTransformerRecognizer.load_from_checkpoint(v_15_ckpt,
                                                            backbone=model, 
                                                            map_location='cpu').backbone

In [ ]:
model_v16 = SwipeTransformerRecognizer.load_from_checkpoint(v_16_ckpt,
                                                            backbone=model, 
                                                            map_location='cpu').backbone

In [ ]:
model_2 = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.3/lightning_logs/version_50464755/checkpoints/epoch=0-step=1000.ckpt.b",
                                                            backbone=model, 
                                                            map_location='cpu').backbone

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.3/lightning_logs/version_50464755/checkpoints/epoch=0-step=1000.ckpt.b",
                                                            backbone=model, 
                                                            map_location='cpu')

In [ ]:
#epoch=0-step=2000.ckpt
#!cp "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt" exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b",
                                                            backbone=model, 
                                                            map_location='cpu')

In [ ]:
module_v7 = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b",
                                                            backbone=model, 
                                                            map_location='cpu').backbone

In [ ]:
def average(model1, model2, w=[0.5, 0.5]):
    model_aver = deepcopy(model1)
    state_dict2 = model2.state_dict()
    for full_param_name, param in model_aver.named_parameters():
        param.data = param.data*w[0] + state_dict2[full_param_name] * w[1]
    return model_aver
model_aver = average(model_v16, pl_module.backbone)

In [ ]:
model_aver = average(model_v16, module_v7)

In [ ]:
pl_module.backbone = model_aver

In [ ]:
#pl_module = SwipeTransformerRecognizer(backbone=model_aver)
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.2/lightning_logs/version_50463954/checkpoints/epoch=0-step=20000.ckpt",
                                                            backbone=model, 
                                                            map_location='cpu')
result = trainer.test(pl_module, val_dataloader)
print(result)                  

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.3/lightning_logs/version_50464755/checkpoints/epoch=0-step=18000.ckpt.b",
                                                            backbone=model, 
                                                            map_location='cpu')
result = trainer.test(pl_module, val_dataloader)
print(result)                  

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.2/lightning_logs/version_50463954/checkpoints/epoch=0-step=20000.ckpt.b",
                                                            backbone=model, 
                                                            map_location='cpu')
pl_module2 = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.3/lightning_logs/version_50464755/checkpoints/epoch=0-step=18000.ckpt.b",
                                                            backbone=model, 
                                                            map_location='cpu')
model_aver = average(pl_module.backbone, pl_module2.backbone)
pl_module.backbone = model_aver
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module.backbone = average_state_dicts([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').eval().backbone
    for ckpt in ["exp/models/conformer_v1.2/lightning_logs/version_50463954/checkpoints/epoch=0-step=20000.ckpt.b", 
                "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b",
                "exp/models/conformer_v1.6/lightning_logs/version_50464766/checkpoints/epoch=0-step=5000.ckpt"]
])
#result = trainer.test(pl_module, val_dataloader)
#print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_state_dicts([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').eval().backbone
    for ckpt in ["exp/models/conformer_v1.2/lightning_logs/version_50463954/checkpoints/epoch=0-step=20000.ckpt.b", 
                "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b",
                "exp/models/conformer_v1.6/lightning_logs/version_50464766/checkpoints/epoch=0-step=5000.ckpt"]
]))
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.5/lightning_logs/version_50464765/checkpoints/epoch=0-step=5000.ckpt",
                                                            backbone=model, 
                                                            map_location='cpu')
model_aver = average(model_v15, pl_module.backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver)  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.6/lightning_logs/version_50464766/checkpoints/epoch=0-step=5000.ckpt",
                                                            backbone=model, 
                                                            map_location='cpu')
model_aver = average(model_v15, pl_module.backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver)  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.8/lightning_logs/version_50464785/checkpoints/epoch=0-step=5000.ckpt",
                                                            backbone=model, 
                                                            map_location='cpu')
model_aver = average(model_v15, pl_module.backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver)  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.9/lightning_logs/version_50464786/checkpoints/epoch=0-step=5000.ckpt",
                                                            backbone=model, 
                                                            map_location='cpu')
model_aver = average(model_v15, pl_module.backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver)  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.11/lightning_logs/version_50464880/checkpoints/epoch=0-step=3000.ckpt",
                                                            backbone=model, 
                                                            map_location='cpu')
model_aver = average(model_v15, pl_module.backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver)  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
model = ConformerWithSinPos(feats_dim=37, num_tokens=500, num_decoder_layers=8, num_encoder_layers=8)
model_v15 = SwipeTransformerRecognizer.load_from_checkpoint(v_15_ckpt,
                                                            backbone=model, 
                                                            map_location='cpu').backbone
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/epoch=0-step=5000.ckpt",
                                                            backbone=model, 
                                                            map_location='cpu')
model_aver = average(model_v15, pl_module.backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver)  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
model = ConformerWithSinPos(feats_dim=37, num_tokens=500, num_decoder_layers=8, num_encoder_layers=8)
model_v15 = SwipeTransformerRecognizer.load_from_checkpoint(v_15_ckpt,
                                                            backbone=model, 
                                                            map_location='cpu').backbone
pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt",
                                                            backbone=model_v15, 
                                                            map_location='cpu')
model_aver = average(model_v15, pl_module.backbone)
#model_aver = average(model_aver, pl_module.backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver)  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
def average_many(models, ws=None):
    if ws is None:
        ws = [1/len(models) for _ in models]
    model_aver = deepcopy(models[0])
    state_dicts = [m.state_dict() for m in models]
    out_state_dict = {}
    for full_param_name, param in model_aver.named_parameters():
        out_state_dict[full_param_name] = sum(p[full_param_name]*w for w, p in zip(ws, state_dicts)) 
        param.data *= ws[0]
        param.data += sum(p[full_param_name]*w for w, p in zip(ws[1:], state_dicts[1:])) 
    #print(f'{out_state_dict.keys()=}')
    #model_aver.load_state_dict(out_state_dict)
    return model_aver

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_many([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').backbone
    for ckpt in [v_15_ckpt]
    
]))  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_many([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').backbone
    for ckpt in ["exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b"]
    
]))  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_many([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').backbone
    for ckpt in [v_15_ckpt, 
                "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b"]
    
]))  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_many([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').backbone
    for ckpt in ["exp/models/conformer_v1.3/lightning_logs/version_50464755/checkpoints/epoch=0-step=1000.ckpt.b", 
                "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b"]
    
]))  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_many([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').backbone
    for ckpt in ["exp/models/conformer_v1.6/lightning_logs/version_50464766/checkpoints/epoch=0-step=5000.ckpt",
                "exp/models/conformer_v1.14/lightning_logs/version_1/checkpoints/epoch=0-step=593.ckpt",
                ]    
]))  
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_many([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').backbone
    for ckpt in [v_15_ckpt, 
                "exp/models/conformer_v1.3/lightning_logs/version_50464755/checkpoints/epoch=0-step=14000.ckpt",
                "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/epoch=0-step=5000.ckpt",
                "exp/models/conformer_v1.9/lightning_logs/version_50464786/checkpoints/epoch=0-step=5000.ckpt",
                "exp/models/conformer_v1.4/lightning_logs/version_50464757/checkpoints/epoch=0-step=5000.ckpt",
                 "exp/models/conformer_v1.2/lightning_logs/version_50463954/checkpoints/epoch=0-step=10000.ckpt.b"
                ]
    
])) 
trainer = pl.Trainer(callbacks=[pl.callbacks.TQDMProgressBar(refresh_rate=100)])
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
model = pl_module.backbone
model.load_state_dict(model.state_dict())

In [ ]:
def average_state_dicts(models, ws=None):
    with torch.no_grad():
        if ws is None:
            ws = [1/len(models) for _ in models]
        state_dicts = [m.state_dict() for m in models]
        out_state_dict = {}
        for full_param_name in state_dicts[0].keys():
            out_state_dict[full_param_name] = sum(p[full_param_name]*w for w, p in zip(ws, state_dicts)) 
       
        #print(f'{out_state_dict.keys()=}')
        model_aver = ConformerWithSinPos(feats_dim=37, num_tokens=500, num_decoder_layers=8, num_encoder_layers=8)
        model_aver.load_state_dict(out_state_dict)
    return model_aver

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=average_state_dicts([
    SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu').eval().backbone
    for ckpt in [v_15_ckpt, 
                "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b"]
]))
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=model) 

pl_module.backbone = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/conformer_v1/lightning_logs/version_50454211/checkpoints/last.ckpt',
                                                    backbone=model, 
                                                    map_location='cpu')
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_module = SwipeTransformerRecognizer(backbone=model) 

pl_module.backbone = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/conformer_v1/lightning_logs/version_50454211/checkpoints/last.ckpt',
                                                    backbone=model, 
                                                    map_location='cpu')
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
pl_modules = [SwipeTransformerRecognizer.load_from_checkpoint(ckpt,
                                                    backbone=model, 
                                                    map_location='cpu')
    for ckpt in [v_15_ckpt, 
                "exp/models/conformer_v1.7/lightning_logs/version_50464775/checkpoints/last.ckpt.b"]
]

for p in pl_modules:
    result = trainer.test(p, val_dataloader)
    print(result)

model_aver = average(pl_modules[0].backbone, pl_modules[1].backbone)
pl_module = SwipeTransformerRecognizer(backbone=model_aver) 
result = trainer.test(pl_module, val_dataloader)
print(result)

In [ ]:
result = trainer.test(pl_module, val_dataloader)
print(result)
# [{'test_loss': 0.15845882892608643}]
# 'test_loss': 0.14765197038650513 - model_v15
# 'test_loss': 0.14764617383480072 - conformer_v1.3 - epoch=0-step=1000.ckpt.b
# 'test_loss':  0.14889037609100342 - conformer_v1.7 - epoch=0-step=2000.ckpt # 0.14889037609100342
# [{'test_loss': 0.13205789029598236}] - model_v15 + conformer_v1.7.2000 - submit_v16
# {'test_loss': 0.1533224731683731} - train_conformer_v1.12.py
# {'test_loss': 0.1533845216035843} - model_v15 + conformer_v1.3.4000
# {'test_loss': 0.1480194628238678} - model_v15 + conformer_v1.4.5000
# 'test_loss': 0.147256538271904 - model_v15 + conformer_v1.5.5000
# {'test_loss': 0.1440354734659195} - model_v15 + conformer_v1.6.5000
# 'test_loss': 0.14874745905399323 - model_v15 + conformer_v1.8.5000

In [ ]:
import sentencepiece as spm
import math
from collections import defaultdict
tokenizer = spm.SentencePieceProcessor('exp/bpe500/model.model')


In [ ]:
topk=20

In [ ]:
utt2words, utt2logs = pl_module.cuda().predict_topk(val_dataloader, tokenizer=tokenizer, topk=topk, device='cuda')

In [ ]:
def accuracy(ref_u2w, hyp_u2w):
    corr = 0
    err = 0
    total = len(ref_u2w)
    for u, ref in tqdm(ref_u2w.items()):
        hyp = hyp_u2w[u].strip('-')
        if ref != hyp:
            print(ref, hyp)
            err +=1
        else:
            corr +=1
    a = corr/total
    print(f"{total=} {corr=} {err=}, accuracy: {a}")
    return a

with open('data_feats/valid/text') as f:
    valid_ref_u2w = {u:w for u, w in   map(str.split, f.readlines())}
    

In [ ]:
with open('./data/voc.txt') as f:
    vocab = frozenset(s for s in map(str.strip, f.readlines()))

In [ ]:
def limit_vocab(u2w, vocab=vocab):
    lv = {}
    for k, v in u2w.items():
        corr_w = []
        for w in v:
            if w in vocab:
                corr_w.append(w)
        if len(corr_w) == 0: 
            logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
            corr_w = ['-']
        lv[k] = corr_w
    return lv
#utt2words_lv = limit_vocab(utt2words)

In [ ]:
print(accuracy(valid_ref_u2w, {k:v[0] for k,v in utt2words.items()}))


In [ ]:
print(accuracy(valid_ref_u2w, {k:v[0] for k,v in utt2words_lv.items()}))
# v15 - total=10000 corr=8887 err=1113, accuracy: 0.8887


In [ ]:
test_ds =  FeatsIterableDatasetV2([f"ark:data_feats/test/feats.ark"], shuffle=False, 
                                 bos_id=1, 
                                 eos_id=2, 
                                 batch_first=False)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, collate_fn=test_ds.collate_pad)
#test_u2w = predict(pl_module.backbone, test_dataloader)
#test_u2w, test_u2l = pl_module.cuda().predict_topk(test_dataloader, tokenizer=tokenizer, topk=topk)

In [ ]:
test_lv = limit_vocab(test_u2w)

In [ ]:
baseline_result = pd.read_csv('./keyboard_start/result/baseline.csv', sep=',', names=['main', 'second', 'third', 'trash'])
#baseline_result = pd.read_csv('exp/models/ctc_trans/lightning_logs/version_50422251/test_submit.v1.csv', sep=',', names=['main', 'second', 'third', 'trash'])
#baseline_result = 
baseline_result['uid'] = [f'test-{i}' for i in range(len(baseline_result))]
baseline_result.head()

baseline_result['predict'] = baseline_result.uid.apply(lambda x: test_lv[x])
baseline_result.head()
rows = []
for i, row in baseline_result.iterrows():
    ps = row['predict']
    for p in [row['main'], row['second'], row['third'], row['trash']]:
        if p not in ps and p != '-':
            ps.append(p)
    rows.append(ps[:4])
        
submission = pd.DataFrame(rows, columns=['main', 'second', 'third', 'trash'])
submission.to_csv("exp/test_submit.v16.csv", 
                  sep=',', header=False, index=False)
submission.head()

In [ ]:
exp_dir='exp'
v="v16"
s5_df = pd.read_csv(f'{exp_dir}/test_submit.{v}.csv', sep=',', names=['main', 'second', 'third', 'trash'])
s5_df['uid'] = [f'test-{i}' for i in range(len(s5_df))]
s5_df.head()

weak_sup_test = {row['uid']: row['main'] if row['main']!= '-' else row['second'] for i, row in s5_df.iterrows()}

with open(f'{exp_dir}/{v}.test.text', 'w') as f:
    f.write(''.join(f"{u} {t}\n" for u, t in weak_sup_test.items()))

encoded = {u: tokenizer.encode(v, out_type="immutable_proto") for u, v in weak_sup_test.items()}
with open(f"{exp_dir}/{v}.test.piece", "w") as f:
    f.write(''.join([f"{u} " + " ".join(e.piece for e in line.pieces) + '\n' for u, line in encoded.items()]))
with open(f"{exp_dir}/{v}.test.int", "w") as f:
    f.write(''.join([f"{u} " + " ".join(str(e.id) for e in line.pieces) + '\n' for u, line in encoded.items()]))


In [ ]:
# test adapted model

adapted_model = SwipeTransformerRecognizer.load_from_checkpoint(
    "exp/models/conformer_v1.14/lightning_logs/version_1/checkpoints/epoch=0-step=593.ckpt",
                                                    backbone=model, 
                                                    map_location='cpu')

test_u2w, test_u2l = adapted_model.cuda().predict_topk(test_dataloader, tokenizer=tokenizer, topk=topk)


In [ ]:
test_lv = limit_vocab(test_u2w)

In [ ]:
baseline_result = pd.read_csv(f'./exp/test_submit.v16.csv', sep=',', names=['main', 'second', 'third', 'trash'])
#baseline_result = pd.read_csv('exp/models/ctc_trans/lightning_logs/version_50422251/test_submit.v1.csv', sep=',', names=['main', 'second', 'third', 'trash'])
#baseline_result = 
baseline_result['uid'] = [f'test-{i}' for i in range(len(baseline_result))]
baseline_result.head()

baseline_result['predict'] = baseline_result.uid.apply(lambda x: test_lv[x])
baseline_result.head()
rows = []
for i, row in baseline_result.iterrows():
    ps = row['predict']
    for p in [row['main'], row['second'], row['third'], row['trash']]:
        if p not in ps and p != '-':
            ps.append(p)
    rows.append(ps[:4])
        
submission = pd.DataFrame(rows, columns=['main', 'second', 'third', 'trash'])
submission.to_csv("exp/test_submit.v17.csv", 
                  sep=',', header=False, index=False)